In [28]:
import pandas as pd

C = pd.read_stata(
    "/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/00_source_data/0405_sec_c0.dta"
)
print(C.shape)

(91319, 21)


In [30]:
C.columns

Index(['hhcode', 'sec', 'msno', 'scq01', 'scq02', 'scq03', 'scqo4', 'scq05',
       'scq06', 'scq07', 'scq08a', 'scq08b', 'scq09a', 'scq09b', 'psu',
       'quarter', 'region', 'psu_hies', 'hhno', 'hhcode_hies', 'province'],
      dtype='object')

In [31]:
# Reduce to the questions of interest (4 questions):

C["ever_admitted"] = C["scq03"]
C["max_level_acheived"] = C["scqo4"]
C["currently_enrrolled"] = C["scq05"] # IMPORTANT!!! (counts of enrrollment)
C["why_not"] = C["scq09a"]

C_relevant = C[
    [
        "hhcode",
        "msno",
        "ever_admitted",
        "max_level_acheived",
        "currently_enrrolled",
        "why_not",
        "region",
        "province"
    ]
].copy()


In [32]:
C_relevant

hhcode  msno ever_admitted max_level_acheived  \
0      1.001100e+09     1           yes           class 10   
1      1.001100e+09     2           yes           cl;ass 8   
2      1.001100e+09     3           yes         17: ma/msc   
3      1.001100e+09     4           yes           class 10   
4      1.001100e+09     1           yes         12: ba/bsc   
...             ...   ...           ...                ...   
91314  4.262002e+09     6            no                NaN   
91315  4.262002e+09     1            no                NaN   
91316  4.262002e+09     2            no                NaN   
91317  4.262002e+09     3            no                NaN   
91318  4.262002e+09     4            no                NaN   

      currently_enrrolled                      why_not region     province  
0                      no        2 education completed  urban       punjab  
1                      no        2 education completed  urban       punjab  
2                      no        2 education completed  urban       punjab  
3                     yes                          NaN  urban       punjab  
4                      no                10 employment  urban       punjab  
...                   ...                          ...    ...          ...  
91314                 NaN                     15 other  rural  balochistan  
91315                 NaN                10 employment  rural  balochistan  
91316                 NaN  13 parent   did not approve  rural  balochistan  
91317                 NaN  13 parent   did not approve  rural  balochistan  
91318                 NaN  13 parent   did not approve  rural  balochistan  

[91319 rows x 8 columns]

In [33]:
B = pd.read_stata(
    "/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/00_source_data/0405_sec_b0.dta"
)
B.sample(5)

# Change the names of the questions for readability
B['sex'] = B['sbq01']
B['age'] = B['sbq04']
B['marital_status'] = B['sbq05']

B_relevant = B[['hhcode', 'msno', 'age', 'sex', 'marital_status']].copy()

In [34]:
B.sample(5)

hhcode sec  msno   sbq01    sbq02                         sbq03  \
8660   1.082002e+09   B     1    male  present                        1:self   
98107  4.191001e+09   B     6    male  present                3:son/daughter   
59272  2.161102e+09   B     6  female  present  4:grand son / grand daughter   
32458  1.262002e+09   B     6    male  present  4:grand son / grand daughter   
23760  1.191002e+09   B     5    male  present                3:son/daughter   

       sbq04               sbq05       weight       psu  quarter region  \
8660      42  4:widow / widowers  2694.220947  10820020        1  rural   
98107     13     1:never married    91.904999  41910005        1  urban   
59272     12     1:never married  1662.515015  21611023        2  urban   
32458     10     1:never married  1309.857056  12620020        2  rural   
23760     10     1:never married  1440.605957  11910023        4  urban   

       psu_hies  hhno   hhcode_hies     province     sex  age  \
8660   10820101    10  1.082010e+09       punjab    male   42   
98107  40510102    10  4.051010e+09  balochistan    male   13   
59272  20511202     3  2.051120e+09        sindh  female   12   
32458  12620202     3  1.262020e+09       punjab    male   10   
23760  10510402     9  1.051040e+09       punjab    male   10   

           marital_status  
8660   4:widow / widowers  
98107     1:never married  
59272     1:never married  
32458     1:never married  
23760     1:never married

In [35]:
# Made the Household Code column a str type
B["hhcode"] = B["hhcode"].astype("str")
C["hhcode"] = C["hhcode"].astype("str")

In [36]:
# MERGE
years04_05 = pd.merge(B_relevant, C_relevant, on=["hhcode", 'msno'], how='inner', indicator=False)

# Evaluate merge 
#years04_05._merge.value_counts()

In [37]:
# Select people in school enrollment age (I would say 4-15 years old)

years04_05 = years04_05.loc[(years04_05["age"] > 4)&(years04_05["age"] <= 15)]
years04_05['year']=2004

In [38]:
# Change dtype to string (hhcode)
years04_05["hhcode"] = years04_05["hhcode"].astype("str")

In [40]:
#years04_05['subprovince']=years04_05['hhcode'].str.extract(r'(^w{3})')
years04_05['subprovince code']= years04_05['hhcode'].apply(lambda x: x[0:4])

In [41]:
years04_05["subprovince code"] = years04_05["subprovince code"].astype("int")

In [42]:
# Now we import the stratum name - code data
strat_name = pd.read_csv(
    "/Users/clarissaache/Documents/IDS 701/stratum0405_v2.csv"
)
strat_name.sample(5)

province  subprovince code subprovince
2         Punjab              1022  Rawalpindi
113  Balochistan              4071      Quetta
102       Punjab              1101  Faisalabad
116       Punjab              1031  Faisalabad
100       Punjab              1111  Rawalpindi

In [43]:
# MERGE
years04_05_ = pd.merge(years04_05, strat_name, on=["subprovince code"], how='inner', indicator=True)

# Evaluate merge 
years04_05_._merge.value_counts()

both          25457
left_only         0
right_only        0
Name: _merge, dtype: int64

In [44]:
years04_05_.sample(5)

hhcode  msno  age     sex   marital_status ever_admitted  \
11003  2042001411.0     5   14    male  1:never married           yes   
20715  3222002015.0     5   15  female  1:never married            no   
9617   1342003406.0     3   10    male  1:never married           yes   
1248   1051100501.0     3   13    male  1:never married           yes   
23251  4102000414.0     7    5  female  1:never married           yes   

        max_level_acheived currently_enrrolled                  why_not  \
11003             cl;ass 8                 yes                      NaN   
20715                  NaN                 NaN  6 had to help with work   
9617               class 3                 yes                      NaN   
1248              cl;ass 8                 yes                      NaN   
23251  0: less than class1                 yes                      NaN   

      region   province_x  year  subprovince code   province_y  \
11003  rural        sindh  2004              2042        Sindh   
20715  rural         nwfp  2004              3222        NWFP    
9617   rural       punjab  2004              1342       Punjab   
1248   urban       punjab  2004              1051       Punjab   
23251  rural  balochistan  2004              4102  Balochistan   

           subprovince _merge  
11003  Nowshero Freoze   both  
20715      LakkiMarwat   both  
9617    Rahim Yar Khan   both  
1248            Lahore   both  
23251          Mastung   both

In [45]:
years04_05_.to_csv('/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/10_data_cleaning/clean_clarissa/years04_05_.', index_label="id")